<a href="https://colab.research.google.com/github/Sam-Davies-Vet-Ai/Data-access/blob/main/BQ_Accsess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.oauth2 import service_account
from googleapiclient import discovery
from google.cloud import bigquery
import json

In [ ]:
# Used for formatting text bold and underlined
start = "\033[1m"
end = "\033[0;0m"
under = "\033[4m"

In [ ]:
#opening the json file with credentials the open('xxxxx.json') needs to be the link to where the file is stored
with open('xxxxx.json') as source:
   gcp_sa_credentials = json.load(source) 

In [ ]:
#Extracts the project id from the credentials
project_id=gcp_sa_credentials["project_id"]

In [ ]:
#Creates the bigquery client connection to the project
credentials = service_account.Credentials.from_service_account_info(gcp_sa_credentials)
client = bigquery.Client(project=project_id,credentials=credentials)

In [ ]:
# function to loop through the tables in the dataset, name the tables and print a dataframe for the first 5 rows
def get_data_extract(datasetname,project_id):

  #Connects to the dataset
  dataset_ref = client.dataset(datasetname, project=project_id)
  dataset = client.get_dataset(dataset_ref)

  # Lists the tables in the dataset
  tables = list(client.list_tables(dataset))

  #Tables title
  print("\n", start + "Tables" + end)

  #Prints the name of each of the tables
  for table in tables:
    print(table.table_id)

  #first 5 rows title
  print("\n",start + "First 5 rows from each table" + end)

  #Loops through tables and prints the first 5 rows
  for table in tables:
    # Prints the name of the table
    print(under + table.table_id + end)

    #Connects to the table
    table_ref = dataset_ref.table(table.table_id)
    select_table = client.get_table(table_ref)
    
    # creates a dataframe of the first 5 rows of the table
    df = client.list_rows(select_table, max_results=5).to_dataframe()

    # prints the dataframe
    print(df,"\n")

In [ ]:
#Listing all of the datasets
datasets = list(client.list_datasets())

#Loops through the dataset and then calls the function get_data_extract to state table names and dataframe extract
for dataset in datasets:

  #Dataset Title
  print("\n", start + "Dataset" + end)

  #Print dataset name
  print(dataset.dataset_id)

  #Call function
  get_data_extract(dataset.dataset_id,project_id)